In [1]:
import numpy as np
import glob
import os
from models.JetPointNet import PointNetSegmentation


def load_data_from_npz(npz_file):
    data = np.load(npz_file)
    feats = data['feats']  # Shape: (num_samples, 859, 6)
    frac_labels = data['frac_labels']  # Shape: (num_samples, 859)
    tot_labels = data['tot_labels']  # Shape: (num_samples, 859)
    tot_truth_e = data['tot_truth_e']  # Shape: (num_samples, 859) (This is the true total energy deposited by particles into this cell)
    return feats, frac_labels, tot_labels, tot_truth_e

# Setup
os.environ['CUDA_VISIBLE_DEVICES'] = ""  # Disable GPU
model_path = "saved_model/PointNetModel.keras"

TEST_DIR = '/data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test'

model = PointNetSegmentation(num_points=859, num_classes=1)
model.load_weights(model_path)

npz_files = glob.glob(os.path.join(TEST_DIR, '*.npz'))



2024-04-23 09:33:06.807081: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 09:33:06.809875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 09:33:06.820114: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 09:33:08.058078: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 09:33:19.645199: E tensor

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def plot_energies(predicted_energies, true_energies, coords, types, sample_index):
    fig = plt.figure(figsize=(14, 7))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    ax1.set_title('Predicted Classifications')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax2.set_title('Actual Classifications')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')

    vmax = 1  # Fixed as we now use binary classification

    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=vmax))
    sm.set_array([])

    for i, t in enumerate(types):
        if t == -1:
            continue

        color = 'r' if predicted_energies[i] > 0.5 else 'b'
        marker = 'x' if true_energies[i] > 0.5 else 'o'
        
        ax1.scatter(coords[i, 0], coords[i, 1], coords[i, 2], color=color, marker=marker)
        ax2.scatter(coords[i, 0], coords[i, 1], coords[i, 2], color=color, marker=marker)

    plt.suptitle(f'Sample {sample_index}')
    cbar = fig.colorbar(sm, ax=[ax1, ax2], shrink=0.5, aspect=20)
    cbar.set_label('Classification (0 or 1)')

    plt.show()

# Process each file
for npz_file_idx, npz_file in enumerate(npz_files):
    if npz_file_idx > 1:  # Only process the first two files for demonstration
        break

    feats, frac_labels, tot_labels, tot_truth_e = load_data_from_npz(npz_file)
    segmentation_logits, energies = model.predict(feats)
    segmentation_logits = np.squeeze(segmentation_logits, axis=-1)

    for sample_idx in range(min(10, len(feats))):
        types = feats[sample_idx][:, -1]
        valid_indices = types != -1  # Exclude -1 types for plotting

        coords = feats[sample_idx][valid_indices, :3]
        types_for_plotting = types[valid_indices]

        print(segmentation_logits[sample_idx][valid_indices])
        predicted_classes = np.where(segmentation_logits[sample_idx] > 0, 1, 0)
        true_classes = np.where(frac_labels[sample_idx] > 0.5, 1, 0)

        predicted_energies_eval = predicted_classes * (types == 0)
        true_energies_eval = true_classes * (types == 0)

        tot_predicted = np.sum(predicted_energies_eval)
        tot_true = np.sum(true_energies_eval)

        print("Predicted Count: ", tot_predicted)
        print("True Count: ", tot_true)
        print("Percentage Difference: ", 100 * np.abs(tot_predicted - tot_true) / max(tot_true, 1), "% ")

        # Plot with adjusted classes
        #plot_energies(predicted_classes[valid_indices], true_classes[valid_indices], coords, types_for_plotting, sample_idx)


13/13 [==============================] - 40s 3s/step


IndexError: boolean index did not match indexed array along dimension 0; dimension is 404 but corresponding boolean dimension is 859